# Markowitzの平均・分散モデル

二次計画問題(Quadratic Programming, QP)をCVXOPTを用いて解く．

In [1]:
import datetime
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import pandas_datareader.stooq as stooq

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_stockvalues_tokyo(stockcode, start, end, use_ratio=False):
    # Get index data from https://stooq.com/
    df = stooq.StooqDailyReader(f"{stockcode}.jp", start, end).read()
    df = df.sort_values(by='Date',ascending=True)
    
    if use_ratio:
        df = df.apply(lambda x: (x - x[0]) / x[0] )
    return df

In [3]:
def get_paneldata_tokyo(stockcodes, start, end, use_ratio=False):
    dfs=[]
    for sc in stockcodes:
        df = get_stockvalues_tokyo(sc, start, end, use_ratio)[['Close']]
        df = df.rename(columns={'Close': sc})
        dfs.append(df)
    df_concat = pd.concat(dfs, axis=1)
    return df_concat

TOPIX 500の全銘柄

https://www.jpx.co.jp/files/tse/news/17/b7gje6000002vm4e-att/meigara-201210-j.pdf

In [4]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2020, 11, 30)
stockcodes=["1301", "1762", "1820", "1967", "2127"]

df = get_paneldata_tokyo(stockcodes, start, end, use_ratio=True)
df

,1301,1762,1820,1967,2127
Date,,,,,
2015-01-05,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-06,-0.010929,-0.018385,-0.033937,-0.002265,-0.038448
2015-01-07,-0.014564,-0.020433,-0.059863,-0.013823,-0.059680
2015-01-08,-0.007302,-0.016338,-0.057883,-0.013823,-0.039787
2015-01-09,0.000000,-0.004490,-0.031938,-0.025407,-0.043770
...,...,...,...,...,...
2020-10-29,0.096138,-0.032923,-0.030777,0.858573,5.682321
2020-10-30,0.093336,-0.039657,-0.041199,0.832831,5.704266
2020-11-02,0.107748,-0.026188,-0.032198,0.845702,5.418978


In [5]:
# Covariance matrix
df.cov()

,1301,1762,1820,1967,2127
1301,0.024211,0.015340,0.018243,0.037772,0.081221
1762,0.015340,0.014867,0.015562,0.023735,0.038868
1820,0.018243,0.015562,0.025023,0.029918,0.040811
1967,0.037772,0.023735,0.029918,0.109754,0.312827
2127,0.081221,0.038868,0.040811,0.312827,1.703412


In [6]:
# expected returns
df.mean().values

array([0.12547322, 0.10879767, 0.07469455, 0.44782516, 1.75209493])

*Markowitzの平均・分散モデル

- $\Sigma  \in \mathbb{R}^{n \times n}$: 共分散行列
- $r  \in \mathbb{R}^{n}$: 期待収益率ベクトル
- $r_i \in \mathbb{R}$: 資産$i$の期待収益率
- $r_e \in \mathbb{R}$: 投資家の要求期待収益率
- $x \in \mathbb{R}^{n}$: 投資比率ベクトル
- $x_i \in \mathbb{R}$: 資産$i$の投資比率


$$
\begin{align}
{\rm min} ~~~ x^T \Sigma x \\
{\rm s.t.} ~~~ r^T x \geq r_e \\
{\| x \|}_{1} = 1 \\
x_i \geq 0
\end{align}
$$

*二次計画問題

![img0](https://cdn-ak.f.st-hatena.com/images/fotolife/m/meison_amsl/20161016/20161016122314.png)


参考:
- https://cvxopt.org/userguide/coneprog.html
- https://qiita.com/ryoshi81/items/8b0c6add3e367f94c828

In [7]:
import cvxopt

def cvxopt_qp_solver(r, r_e, cov):
    P = cvxopt.matrix((1/2) * cov)
    q = cvxopt.matrix(np.zeros(len(r)))
    G = cvxopt.matrix(-1.0 * np.diag(r))
    h = cvxopt.matrix(-1.0 * r_e)
    A = cvxopt.matrix(1.0, (1, len(r)))
    b = cvxopt.matrix(1.0)
    sol=cvxopt.solvers.qp(P,q,G,h,A,b)
    return sol

In [8]:
r = df.mean().values
r_e = np.zeros(df.mean().values.shape)
cov = np.array(df.cov())

sol = cvxopt_qp_solver(r, r_e, cov)
print(sol["x"])
print("total return: ", sol["primal objective"])

     pcost       dcost       gap    pres   dres
 0:  4.3680e-03 -8.6883e-02  5e+00  2e+00  2e+00
 1:  9.1180e-02 -2.2275e-01  5e-01  1e-01  1e-01
 2:  2.1337e-02 -6.0274e-02  8e-02  2e-16  1e-16
 3:  1.0483e-02 -1.7810e-03  1e-02  1e-16  3e-17
 4:  4.9857e-03  1.5180e-03  3e-03  2e-16  8e-18
 5:  4.0217e-03  3.6059e-03  4e-04  3e-17  1e-17
 6:  3.7560e-03  3.7107e-03  5e-05  3e-17  1e-18
 7:  3.7187e-03  3.7168e-03  2e-06  1e-17  4e-18
 8:  3.7169e-03  3.7168e-03  2e-08  1e-16  6e-18
Optimal solution found.
[ 5.56e-05]
[ 1.00e+00]
[ 1.76e-05]
[ 3.84e-07]
[ 2.63e-07]

total return:  0.003716866155475511
